# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 19 2023 9:44AM,264946,15,30017271,"Alliance Pharma, Inc.",Released
1,Jun 19 2023 9:44AM,264946,15,30017270,"Alliance Pharma, Inc.",Released
2,Jun 19 2023 9:44AM,264946,15,30017272,"Alliance Pharma, Inc.",Released
3,Jun 19 2023 9:44AM,264946,15,30017273,"Alliance Pharma, Inc.",Released
4,Jun 19 2023 9:44AM,264946,15,30017274,"Alliance Pharma, Inc.",Released
5,Jun 19 2023 9:44AM,264946,15,30017275,"Alliance Pharma, Inc.",Released
6,Jun 19 2023 9:44AM,264946,15,30017319,"Alliance Pharma, Inc.",Released
7,Jun 19 2023 9:44AM,264946,15,30017318,"Alliance Pharma, Inc.",Released
8,Jun 19 2023 9:44AM,264946,15,30017311,"Alliance Pharma, Inc.",Released
9,Jun 19 2023 9:44AM,264946,15,30017313,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,264940,Released,3
19,264941,Released,1
20,264942,Released,6
21,264943,Released,5
22,264946,Released,50


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264940,NaN,NaN,3.0
264941,NaN,NaN,1.0
264942,NaN,NaN,6.0
264943,NaN,NaN,5.0
264946,NaN,NaN,50.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264921,1.0,2.0,1.0
264925,0.0,1.0,0.0
264926,0.0,16.0,10.0
264927,0.0,1.0,0.0
264928,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264921,1,2,1
264925,0,1,0
264926,0,16,10
264927,0,1,0
264928,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264921,1,2,1
1,264925,0,1,0
2,264926,0,16,10
3,264927,0,1,0
4,264928,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264921,1,2,1
1,264925,,1,
2,264926,,16,10
3,264927,,1,
4,264928,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 19 2023 9:44AM,264946,15,"Alliance Pharma, Inc."
50,Jun 19 2023 9:28AM,264943,19,"Person & Covey, Inc."
55,Jun 19 2023 9:20AM,264942,10,"Snap Medical Industries, LLC"
61,Jun 19 2023 9:19AM,264941,10,"Snap Medical Industries, LLC"
62,Jun 19 2023 9:17AM,264940,10,Emerginnova
65,Jun 19 2023 9:16AM,264939,10,"Methapharm, Inc."
76,Jun 19 2023 9:14AM,264938,10,"Methapharm, Inc."
86,Jun 19 2023 9:10AM,264937,15,"Brookfield Pharmaceuticals, LLC"
102,Jun 19 2023 9:09AM,264936,10,"Methapharm, Inc."
103,Jun 19 2023 9:04AM,264935,19,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 19 2023 9:44AM,264946,15,"Alliance Pharma, Inc.",,,50
1,Jun 19 2023 9:28AM,264943,19,"Person & Covey, Inc.",,,5
2,Jun 19 2023 9:20AM,264942,10,"Snap Medical Industries, LLC",,,6
3,Jun 19 2023 9:19AM,264941,10,"Snap Medical Industries, LLC",,,1
4,Jun 19 2023 9:17AM,264940,10,Emerginnova,,,3
5,Jun 19 2023 9:16AM,264939,10,"Methapharm, Inc.",,,11
6,Jun 19 2023 9:14AM,264938,10,"Methapharm, Inc.",,2,8
7,Jun 19 2023 9:10AM,264937,15,"Brookfield Pharmaceuticals, LLC",,12,4
8,Jun 19 2023 9:09AM,264936,10,"Methapharm, Inc.",,,1
9,Jun 19 2023 9:04AM,264935,19,"Exact-Rx, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 9:44AM,264946,15,"Alliance Pharma, Inc.",50,,
1,Jun 19 2023 9:28AM,264943,19,"Person & Covey, Inc.",5,,
2,Jun 19 2023 9:20AM,264942,10,"Snap Medical Industries, LLC",6,,
3,Jun 19 2023 9:19AM,264941,10,"Snap Medical Industries, LLC",1,,
4,Jun 19 2023 9:17AM,264940,10,Emerginnova,3,,
5,Jun 19 2023 9:16AM,264939,10,"Methapharm, Inc.",11,,
6,Jun 19 2023 9:14AM,264938,10,"Methapharm, Inc.",8,2,
7,Jun 19 2023 9:10AM,264937,15,"Brookfield Pharmaceuticals, LLC",4,12,
8,Jun 19 2023 9:09AM,264936,10,"Methapharm, Inc.",1,,
9,Jun 19 2023 9:04AM,264935,19,"Exact-Rx, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 9:44AM,264946,15,"Alliance Pharma, Inc.",50,,
1,Jun 19 2023 9:28AM,264943,19,"Person & Covey, Inc.",5,,
2,Jun 19 2023 9:20AM,264942,10,"Snap Medical Industries, LLC",6,,
3,Jun 19 2023 9:19AM,264941,10,"Snap Medical Industries, LLC",1,,
4,Jun 19 2023 9:17AM,264940,10,Emerginnova,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 9:44AM,264946,15,"Alliance Pharma, Inc.",50.0,NaN,NaN
1,Jun 19 2023 9:28AM,264943,19,"Person & Covey, Inc.",5.0,NaN,NaN
2,Jun 19 2023 9:20AM,264942,10,"Snap Medical Industries, LLC",6.0,NaN,NaN
3,Jun 19 2023 9:19AM,264941,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
4,Jun 19 2023 9:17AM,264940,10,Emerginnova,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 9:44AM,264946,15,"Alliance Pharma, Inc.",50.0,0.0,0.0
1,Jun 19 2023 9:28AM,264943,19,"Person & Covey, Inc.",5.0,0.0,0.0
2,Jun 19 2023 9:20AM,264942,10,"Snap Medical Industries, LLC",6.0,0.0,0.0
3,Jun 19 2023 9:19AM,264941,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
4,Jun 19 2023 9:17AM,264940,10,Emerginnova,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1854567,38.0,12.0,0.0
15,529883,54.0,12.0,0.0
19,1589584,18.0,18.0,0.0
20,529855,16.0,2.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1854567,38.0,12.0,0.0
1,15,529883,54.0,12.0,0.0
2,19,1589584,18.0,18.0,0.0
3,20,529855,16.0,2.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,38.0,12.0,0.0
1,15,54.0,12.0,0.0
2,19,18.0,18.0,0.0
3,20,16.0,2.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,38.0
1,15,Released,54.0
2,19,Released,18.0
3,20,Released,16.0
4,10,Executing,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,0.0,0.0,0.0,1.0
Executing,12.0,12.0,18.0,2.0
Released,38.0,54.0,18.0,16.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,0.0,0.0,0.0,1.0
1,Executing,12.0,12.0,18.0,2.0
2,Released,38.0,54.0,18.0,16.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,0.0,0.0,0.0,1.0
1,Executing,12.0,12.0,18.0,2.0
2,Released,38.0,54.0,18.0,16.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()